In [1]:
# Visualization------------------------------------ Radiomcis feature selection
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import ElasticNetCV, ElasticNet
import matplotlib
import warnings

warnings.filterwarnings("ignore")

def plot_elastic_net_path_vertical_no_block(X, y, feature_names, l1_ratio=0.5, n_alphas=100, cv=5):
    """
    Main plot and legend are arranged vertically.
    The legend is manually formatted as text and does not block the axes.
    """
    eps = 1e-3
    alphas = np.logspace(np.log10(eps), np.log10(1), n_alphas)
    enet_cv = ElasticNetCV(
        l1_ratio=l1_ratio,
        alphas=alphas,
        cv=cv,
        random_state=42,
        max_iter=1000
    )
    enet_cv.fit(X, y)

    coef_path = []
    enet = ElasticNet(l1_ratio=l1_ratio, max_iter=1000)
    for alpha in alphas:
        enet.set_params(alpha=alpha)
        enet.fit(X, y)
        coef_path.append(enet.coef_)
    coef_path = np.array(coef_path)

    fig = plt.figure(figsize=(10, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[1.1, 0.9], hspace=0.10)
    ax = fig.add_subplot(gs[0, 0])

    # All paths in gray
    ax.plot(np.log10(alphas), coef_path, color='gray', linewidth=0.5, alpha=0.1)

    final_coef = enet_cv.coef_
    nonzero_idx = abs(final_coef) > 1e-5
    colors = plt.cm.tab10(np.linspace(0, 1, sum(nonzero_idx)))

    # Highlighted features
    feature_lines = []
    coef_idx = 0
    for idx, (is_selected, coef) in enumerate(zip(nonzero_idx, final_coef)):
        if is_selected:
            line = ax.plot(np.log10(alphas), coef_path[:, idx],
                           linewidth=2, color=colors[coef_idx],
                           label=f'{feature_names[idx]} ({coef:.4f})')[0]
            feature_lines.append((line, f'{feature_names[idx]} ({coef:.4f})'))
            coef_idx += 1

    # Optimal alpha line
    optimal_line = ax.axvline(np.log10(enet_cv.alpha_), color='r', linestyle='--', alpha=0.8)
    feature_lines.append((optimal_line, f'Optimal α = {enet_cv.alpha_:.2e}'))

    ax.set_xlabel('log(α)', fontsize=16)
    ax.set_ylabel('Coefficients', fontsize=24)
    ax.grid(True, alpha=0.1)
    ax.set_title('Elastic Net Path', fontsize=24, pad=20)

    # Lower subplot for legend, center-aligned text only
    ax_leg = fig.add_subplot(gs[1, 0])
    ax_leg.axis('off')

    # Legend text content
    legend_texts = [f"{line[1]}" for line in feature_lines]
    model_info = [
        f"Total features: {X.shape[1]}",
        f"Selected features: {sum(nonzero_idx)}",
        f"Optimal α: {enet_cv.alpha_:.2e}",
        f"l1_ratio: {l1_ratio:.2f}",
        f"Mean CV score: {enet_cv.score(X, y):.4f}"
    ]
    # Colored small squares
    y_start = 1.0
    line_height = 0.09
    for i, (line, label) in enumerate(feature_lines):
        color = line.get_color()
        # Colored rectangle
        ax_leg.plot([0.02, 0.07], [y_start - i*line_height]*2, color=color, linewidth=6, solid_capstyle='round')
        ax_leg.text(0.10, y_start - i*line_height, label, va='center', fontsize=20)

    # Optimal alpha dashed line
    ax_leg.plot([0.02, 0.07], [y_start - (len(feature_lines)-1)*line_height]*2, color='red', linestyle='--', linewidth=3)
    ax_leg.text(0.10, y_start - (len(feature_lines)-1)*line_height, feature_lines[-1][1], va='center', fontsize=20, color='red')

    # Model info
    for j, info in enumerate(model_info):
        ax_leg.text(0.02, y_start - (len(feature_lines)+0.5+j)*line_height, info, va='center', fontsize=20)

    ax_leg.set_xlim(0, 1)
    ax_leg.set_ylim(0, 1.05)

    plt.tight_layout()
    return enet_cv, final_coef

def visualize_elastic_net_selected(model, feature_names, save_path=None):
    """
    Visualize only features selected by Elastic Net (nonzero coefficients) and their coefficients
    """
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    mpl.rcParams['font.family'] = 'Arial'
    mpl.rcParams['font.size'] = 16
    plt.figure(figsize=(10, max(4, 0.6*sum(abs(model.coef_) > 1e-5))))
    
    coefficients = model.coef_.flatten()
    # Select only nonzero coefficient features
    selected = np.abs(coefficients) > 1e-5
    selected_coefs = coefficients[selected]
    selected_features = np.array(feature_names)[selected]
    
    # Sort by absolute value
    sorted_indices = np.argsort(np.abs(selected_coefs))[::-1]
    selected_coefs = selected_coefs[sorted_indices]
    selected_features = selected_features[sorted_indices]
    colors = ['#3498db' if coef > 0 else '#e74c3c' for coef in selected_coefs]
    
    plt.barh(range(len(selected_coefs)), selected_coefs, color=colors)
    plt.yticks(range(len(selected_coefs)), selected_features, fontsize=18)
    plt.xlabel('Coefficient Value', fontsize=16)
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def main():
    import sys
    data_path = input("Please enter the file path: ").strip()
    if not data_path:
        print("No file path provided. Exiting.")
        sys.exit(1)
    data = pd.read_excel(data_path)
    print(f"Processing file: {data_path}\n")

    X = data.iloc[:, 1:].values
    y = data.iloc[:, 0].values
    original_feature_names = data.columns[1:].tolist()
    print(f"Original number of features: {len(original_feature_names)}")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    var_thresh = VarianceThreshold(threshold=0.01)
    X_train_s = var_thresh.fit_transform(X_train_s)
    X_test_s = var_thresh.transform(X_test_s)
    selected_features_var = var_thresh.get_support()
    var_selected_features = [f for i, f in enumerate(original_feature_names)
                             if selected_features_var[i]]

    print(f"\nAfter Variance Threshold Selection:")
    print(f"Number of features: {len(var_selected_features)}")

    print("\nPlotting Elastic Net path...")
    enet_fitted, final_coefficients = plot_elastic_net_path_vertical_no_block(
        X_train_s,
        y_train,
        var_selected_features,
        l1_ratio=0.5,
        n_alphas=100,
        cv=5
    )
    plt.savefig('elastic_net_path_vertical_no_block.pdf', dpi=300, bbox_inches='tight')
    plt.show()

    print("\nBest parameters:")
    print(f"Optimal alpha: {enet_fitted.alpha_:.2e}")
    print(f"l1_ratio: {enet_fitted.l1_ratio_:.2f}")
    print(f"Number of selected features: {sum(abs(final_coefficients) > 1e-5)}")

    print("\nSelected features and their coefficients:")
    selected_features = pd.DataFrame({
        'Feature': var_selected_features,
        'Coefficient': final_coefficients
    })
    selected_features = selected_features[abs(selected_features['Coefficient']) > 1e-5]
    selected_features = selected_features.sort_values('Coefficient',
                                                      key=abs,
                                                      ascending=False)
    print(selected_features)
    selected_features.to_excel('elastic_net_features.xlsx', index=False)
    # Visualize selected features' coefficients
    print("\nPlotting selected features coefficient bar chart...")
    visualize_elastic_net_selected(enet_fitted, var_selected_features, save_path='elastic_net_selected_coefficients.pdf')

if __name__ == "__main__":
    main()

In [2]:
# Multi-model screening --------------------------- Clinical modal
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.calibration import calibration_curve
import statsmodels.api as sm
import warnings

# Hide warnings in output
warnings.filterwarnings("ignore")

# Load dataset
data_path = input("Please enter the file path: ").strip()
data = pd.read_excel(data_path)
data.describe()

# Separate features (X) and target variable (y)
X = data.iloc[:, 1:].values  # Convert to NumPy array
y = data.iloc[:, 0].values   # Convert to NumPy array

# Get feature names
feature_names = data.columns[1:]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Data standardization
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

# ---------------------------
# Univariate logistic regression for initial feature selection
# ---------------------------
p_values = []
selected_features = []

for i in range(X_train_s.shape[1]):
    X_single = X_train_s[:, i].reshape(-1, 1)
    logit_model = sm.Logit(y_train, sm.add_constant(X_single))
    result = logit_model.fit(disp=0)
    p_value = result.pvalues[1]
    p_values.append(p_value)
    if p_value < 0.05:  # Significance level set at 0.05
        selected_features.append(i)

# Indices of selected features (from univariate logistic regression)
selected_features = np.array(selected_features)

# Show names of initially selected features
print("Initially selected feature names (Univariate Logistic Regression):")
print(feature_names[selected_features])

# ---------------------------
# Further feature selection using Elastic Net
# ---------------------------
if len(selected_features) > 0:
    X_train_selected = X_train_s[:, selected_features]
    X_test_selected = X_test_s[:, selected_features]
    
    # Elastic Net Logistic Regression
    # l1_ratio controls the mix of L1 and L2 regularization, C is the inverse of regularization strength, use saga solver
    elastic_net_model = LogisticRegression(random_state=42, penalty='elasticnet', solver='saga', l1_ratio=0.5, C=1.0, max_iter=5000)
    elastic_net_model.fit(X_train_selected, y_train)
    
    # Get model coefficients (shape: (1, n_features))
    coef = elastic_net_model.coef_.flatten()
    
    # Set a threshold: coefficients with absolute value > threshold are considered important features
    threshold = 1e-4
    elastic_net_selected_indices = np.where(np.abs(coef) > threshold)[0]
    
    # If no features remain after elastic net, retain univariate selection
    if len(elastic_net_selected_indices) == 0:
        multistep_selected_features = selected_features
    else:
        multistep_selected_features = selected_features[elastic_net_selected_indices]
    
    # Show finally selected feature names
    print("Final selected feature names (Elastic Net Regression):")
    print(feature_names[multistep_selected_features])
else:
    print("No features passed univariate logistic regression selection.")
    multistep_selected_features = np.array([])

# If no features remain after screening, exit
if multistep_selected_features.size == 0:
    raise ValueError("No valid features passed selection.")

# Update training and testing sets
X_train_final = X_train_s[:, multistep_selected_features]
X_test_final = X_test_s[:, multistep_selected_features]

# ---------------------------
# Define models
# ---------------------------
base_models = [
    ('lr', LogisticRegression(random_state=42, penalty='l2', C=0.1)),
    ('dt', DecisionTreeClassifier(random_state=42, max_depth=5)),
    ('svm', SVC(probability=True, random_state=42, C=0.1))
]

# Bagging model
bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42, max_depth=5), n_estimators=50, random_state=42)

# Boosting model, AdaBoost with Random Forest as base estimator
boosting_model = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=10, random_state=42, max_depth=5), n_estimators=50, random_state=42)

# Stacking model
stacking_model = StackingClassifier(classifiers=[clf for _, clf in base_models], meta_classifier=LogisticRegression(random_state=42, penalty='l2', C=0.1))

# Rotation Forest model (random forest as replacement)
rotation_forest_model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=5)

# Gradient Boosting model with randomness
gtb_randomness_model = GradientBoostingClassifier(n_estimators=50, random_state=42, subsample=0.8, max_depth=5)

# Model dictionary
models = {
    "Bagging": bagging_model,
    "Boosting": boosting_model,
    "Stacking": stacking_model,
    "Rotation Forest": rotation_forest_model,
    "GTB Randomness": gtb_randomness_model,
}

# ---------------------------
# 5-fold cross-validation AUC on training set
# ---------------------------
kf_train = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}
print("Training set cross-validation AUC:")
for name, model in models.items():
    aucs_train = cross_val_score(model, X_train_final, y_train, cv=kf_train, scoring='roc_auc', n_jobs=-1)
    cv_results[name] = aucs_train
    print(f"{name} - Mean AUC: {aucs_train.mean():.2f} +/- {aucs_train.std():.2f}")

# ---------------------------
# 5-fold cross-validation AUC on testing set
# ---------------------------
kf_test = KFold(n_splits=5, shuffle=True, random_state=42)
print("\nTesting set cross-validation AUC:")
for name, model in models.items():
    aucs_test = cross_val_score(model, X_test_final, y_test, cv=kf_test, scoring='roc_auc', n_jobs=-1)
    print(f"{name} - Mean AUC: {aucs_test.mean():.2f} +/- {aucs_test.std():.2f}")

In [3]:
# Multi-model screening --------------------------- Radiomics modal
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from mlxtend.classifier import StackingClassifier
from sklearn.calibration import calibration_curve
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
import warnings

# Hide warnings in output
warnings.filterwarnings("ignore")

# Load dataset
data_path = input("Please enter the file path: ").strip()
data = pd.read_excel(data_path)
data.describe()

# Separate features (X) and target variable (y)
X = data.iloc[:, 1:].values  # Convert to NumPy array
y = data.iloc[:, 0].values   # Convert to NumPy array

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Data standardization
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

# Initial feature selection (Variance Threshold)
var_thresh = VarianceThreshold(threshold=0.01)
X_train_s = var_thresh.fit_transform(X_train_s)
X_test_s = var_thresh.transform(X_test_s)

# Further feature selection with Elastic Net
elastic_net = ElasticNetCV(cv=5, random_state=42)
select_from_model = SelectFromModel(estimator=elastic_net)
X_train_s = select_from_model.fit_transform(X_train_s, y_train)
X_test_s = select_from_model.transform(X_test_s)

# Define base models
base_models = [
    ('lr', LogisticRegression(random_state=42, penalty='l2', C=0.1)),
    ('dt', DecisionTreeClassifier(random_state=42, max_depth=5)),
    ('svm', SVC(probability=True, random_state=42, C=0.1))
]

# Bagging model
bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42, max_depth=5), n_estimators=50, random_state=42)

# Boosting model with Random Forest as base estimator
boosting_model = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=10, random_state=42, max_depth=5), n_estimators=50, random_state=42)

# Stacking model
stacking_model = StackingClassifier(classifiers=[clf for _, clf in base_models], meta_classifier=LogisticRegression(random_state=42, penalty='l2', C=0.1))

# # Voting model (uncomment if needed)
# voting_model = VotingClassifier(estimators=base_models, voting='soft')

# Rotation Forest model
rotation_forest_model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=5)

# Gradient Boosting model with randomness
gtb_randomness_model = GradientBoostingClassifier(n_estimators=50, random_state=42, subsample=0.8, max_depth=5)

# Initialize models
models = {
    "Bagging": bagging_model,
    "Boosting": boosting_model,
    "Stacking": stacking_model,
    "Rotation Forest": rotation_forest_model,
    "GTB Randomness": gtb_randomness_model,
}

# 5-fold cross-validation and AUC computation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}
for name, model in models.items():
    aucs = cross_val_score(model, X_test_s, y_test, cv=kf, scoring='roc_auc', n_jobs=-1)
    cv_results[name] = aucs
    print(f"{name} - Mean AUC: {aucs.mean():.4f} +/- {aucs.std():.4f}")

In [8]:
# Hyperparameter tuning---------------------------- Clinical modal
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.calibration import calibration_curve
import statsmodels.api as sm
import warnings

# Hide warnings
warnings.filterwarnings("ignore")

# Load dataset
data_path = input("Please enter the file path: ").strip()
data = pd.read_excel(data_path)
data.describe()

# Separate features (X) and target variable (y)
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

# Get feature names
feature_names = data.columns[1:]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Standardize data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

# ---------------------------
# Univariate logistic regression for initial feature selection
# ---------------------------
p_values = []
selected_features = []

for i in range(X_train_s.shape[1]):
    X_single = X_train_s[:, i].reshape(-1, 1)
    logit_model = sm.Logit(y_train, sm.add_constant(X_single))
    result = logit_model.fit(disp=0)
    p_value = result.pvalues[1]
    p_values.append(p_value)
    if p_value < 0.05:  # Significance level
        selected_features.append(i)

# Indices of selected features (univariate logistic regression)
selected_features = np.array(selected_features)

# Display names of initially selected features
print("Initially selected feature names (Univariate Logistic Regression):")
print(feature_names[selected_features])

# ---------------------------
# Further feature selection using Elastic Net
# ---------------------------
if len(selected_features) > 0:
    X_train_selected = X_train_s[:, selected_features]
    X_test_selected = X_test_s[:, selected_features]
    
    # Elastic Net Logistic Regression
    elastic_net_model = LogisticRegression(random_state=42, penalty='elasticnet', 
                                         solver='saga', l1_ratio=0.5, C=1.0, max_iter=5000)
    elastic_net_model.fit(X_train_selected, y_train)
    
    # Get model coefficients
    coef = elastic_net_model.coef_.flatten()
    
    # Set a threshold: coefficients with abs > threshold are important features
    threshold = 1e-4
    elastic_net_selected_indices = np.where(np.abs(coef) > threshold)[0]
    
    # If no features remain after elastic net, retain univariate selection
    if len(elastic_net_selected_indices) == 0:
        multistep_selected_features = selected_features
    else:
        multistep_selected_features = selected_features[elastic_net_selected_indices]
    
    # Display final selected features
    print("\nFinal selected feature names (Elastic Net Regression):")
    print(feature_names[multistep_selected_features])
else:
    print("No features passed univariate logistic regression selection.")
    multistep_selected_features = np.array([])

# Update train/test sets
X_train_final = X_train_s[:, multistep_selected_features]
X_test_final = X_test_s[:, multistep_selected_features]

# ---------------------------
# Grid Search for Parameters
# ---------------------------
def evaluate_model(params, X, y, kf):
    """Evaluate model performance under given parameters"""
    model = RandomForestClassifier(random_state=42, **params)
    scores = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')
    return scores.mean()

# Define parameter grid
param_grid = {
    'n_estimators': [30, 50, 70],           # Number of trees
    'max_depth': [3, 5, 7],                 # Max tree depth
    'min_samples_split': [2, 5, 10],        # Min samples to split node
}

# Store performance for each parameter set
results = []

# Define cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid search
print("\nStart parameter optimization...")
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for min_samples_split in param_grid['min_samples_split']:
            params = {
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split
            }
            
            # 5-fold cross-validation on test set
            mean_auc = evaluate_model(params, X_test_final, y_test, kf)
            
            results.append({
                'params': params,
                'mean_auc': mean_auc
            })
            
            print(f"Parameters: {params}")
            print(f"Mean AUC: {mean_auc:.4f}")
            print("-" * 50)

# Find best parameters
best_result = max(results, key=lambda x: x['mean_auc'])
print("\nBest parameter combination:")
print(f"Parameters: {best_result['params']}")
print(f"AUC: {best_result['mean_auc']:.4f}")

# Visualize grid search results
plt.figure(figsize=(12, 6))
aucs = [r['mean_auc'] for r in results]
param_names = [f"n:{r['params']['n_estimators']},d:{r['params']['max_depth']},s:{r['params']['min_samples_split']}" 
               for r in results]

plt.bar(range(len(results)), aucs)
plt.xticks(range(len(results)), param_names, rotation=45, ha='right')
plt.xlabel('Parameter Combinations')
plt.ylabel('Mean AUC')
plt.title('Parameter Optimization Results')
plt.tight_layout()
plt.savefig('parameter_optimization.pdf')
plt.show()

# ---------------------------
# Use best parameters for Rotation Forest model
# ---------------------------
rotation_forest = RandomForestClassifier(random_state=42, **best_result['params'])

# ---------------------------
# 5-fold cross-validation
# ---------------------------
# On training set
cv_scores_train = cross_val_score(rotation_forest, X_train_final, y_train, 
                                 cv=kf, scoring='roc_auc')
print("\nTraining set 5-fold cross-validation AUC:")
print(f"Mean AUC: {cv_scores_train.mean():.4f} +/- {cv_scores_train.std():.4f}")

# On test set
cv_scores_test = cross_val_score(rotation_forest, X_test_final, y_test, 
                                cv=kf, scoring='roc_auc')
print("\nTest set 5-fold cross-validation AUC:")
print(f"Mean AUC: {cv_scores_test.mean():.4f} +/- {cv_scores_test.std():.4f}")

In [7]:
# Hyperparameter tuning---------------------------- Radiomics modal
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, make_scorer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.calibration import calibration_curve
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.linear_model import ElasticNetCV
import warnings

# Hide warnings in output
warnings.filterwarnings("ignore")

# Load dataset
data_path = input("Please enter the file path: ").strip()
data = pd.read_excel(data_path)

# Separate features (X) and target variable (y)
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Standardize data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

# Initial feature selection (Variance Threshold)
var_thresh = VarianceThreshold(threshold=0.01)
X_train_s = var_thresh.fit_transform(X_train_s)
X_test_s = var_thresh.transform(X_test_s)

# Further feature selection with Elastic Net
elastic_net = ElasticNetCV(cv=5, random_state=42)
select_from_model = SelectFromModel(estimator=elastic_net)
X_train_s = select_from_model.fit_transform(X_train_s, y_train)
X_test_s = select_from_model.transform(X_test_s)

# ---------------------------
# Grid search for parameter tuning
# ---------------------------
def evaluate_model(params, X, y, kf):
    """Evaluate model performance under given parameters"""
    # Set parameters for base RandomForest
    base_rf = RandomForestClassifier(
        n_estimators=10,  # keep base model small
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        random_state=42
    )
    
    # Create AdaBoost model
    model = AdaBoostClassifier(
        estimator=base_rf,
        n_estimators=params['n_estimators'],
        random_state=42
    )
    
    scores = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')
    return scores.mean()

# Define parameter grid
param_grid = {
    'n_estimators': [30, 50, 70],           # AdaBoost iterations
    'max_depth': [3, 5, 7],                 # RandomForest base model tree depth
    'min_samples_split': [2, 5, 10]         # RandomForest base model min samples split
}

# Store performance for each parameter set
results = []

# Define cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid search
print("\nStarting parameter optimization...")
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for min_samples_split in param_grid['min_samples_split']:
            params = {
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split
            }
            
            # 5-fold cross-validation on the test set
            mean_auc = evaluate_model(params, X_test_s, y_test, kf)
            
            results.append({
                'params': params,
                'mean_auc': mean_auc
            })
            
            print(f"Parameters: {params}")
            print(f"Mean AUC: {mean_auc:.4f}")
            print("-" * 50)

# Find the best parameters
best_result = max(results, key=lambda x: x['mean_auc'])
print("\nBest parameter combination:")
print(f"Parameters: {best_result['params']}")
print(f"AUC: {best_result['mean_auc']:.4f}")

# Visualize grid search results
plt.figure(figsize=(12, 6))
aucs = [r['mean_auc'] for r in results]
param_names = [f"n:{r['params']['n_estimators']},d:{r['params']['max_depth']},s:{r['params']['min_samples_split']}" 
               for r in results]

plt.bar(range(len(results)), aucs)
plt.xticks(range(len(results)), param_names, rotation=45, ha='right')
plt.xlabel('Parameter Combinations')
plt.ylabel('Mean AUC')
plt.title('Parameter Optimization Results')
plt.tight_layout()
plt.savefig('boosting_parameter_optimization.pdf')
plt.show()

# Use best parameters to create the final model
best_rf = RandomForestClassifier(
    n_estimators=10,
    max_depth=best_result['params']['max_depth'],
    min_samples_split=best_result['params']['min_samples_split'],
    random_state=42
)

final_boosting_model = AdaBoostClassifier(
    estimator=best_rf,
    n_estimators=best_result['params']['n_estimators'],
    random_state=42
)

# Evaluate the final model
final_scores = cross_val_score(final_boosting_model, X_test_s, y_test, cv=kf, scoring='roc_auc')
print(f"\nFinal model mean AUC: {final_scores.mean():.4f} +/- {final_scores.std():.4f}")

In [ ]:
# Early fusion------------------------------------- Clinical and radiomics modals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier  # Ensure that mlxtend is installed: pip install mlxtend
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

# =============================================================================
# 1. Data Loading and Preprocessing
# =============================================================================
# Load clinical data (modality 1)
clinical_data = pd.read_excel("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
X1 = clinical_data.iloc[:, 1:].values
y1 = clinical_data.iloc[:, 0].values
feature_names1 = clinical_data.columns[1:]

# Load imaging data (modality 2)
imaging_data = pd.read_excel("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
X2 = imaging_data.iloc[:, 1:].values
y2 = imaging_data.iloc[:, 0].values

# Assume labels are consistent; otherwise, sample alignment is needed. Use clinical labels for analysis.
# Split data, ensuring matching sample order for both modalities
X1_train, X1_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, stratify=y1, random_state=42)
X2_train, X2_test, _, _ = train_test_split(X2, y2, test_size=0.3, stratify=y2, random_state=42)

# =============================================================================
# 2. Feature Processing and Selection
# =============================================================================
# ------------------------------
# Clinical feature processing & selection
# ------------------------------
scaler1 = StandardScaler()
X1_train_s = scaler1.fit_transform(X1_train)
X1_test_s = scaler1.transform(X1_test)

# Univariate logistic regression for feature selection
p_values = []
selected_features = []
for i in range(X1_train_s.shape[1]):
    X_single = X1_train_s[:, i].reshape(-1, 1)
    logit_model = sm.Logit(y_train, sm.add_constant(X_single))
    result = logit_model.fit(disp=0)
    p_value = result.pvalues[1]
    p_values.append(p_value)
    if p_value < 0.05:
        selected_features.append(i)
selected_features = np.array(selected_features)

print("\nInitial selected clinical features (univariate LR):")
print(feature_names1[selected_features])

# Elastic Net for further selection
if len(selected_features) > 0:
    X1_train_selected = X1_train_s[:, selected_features]
    X1_test_selected = X1_test_s[:, selected_features]
    elastic_net_model = LogisticRegression(random_state=42, penalty='elasticnet', 
                                           solver='saga', l1_ratio=0.5, C=1.0, max_iter=5000)
    elastic_net_model.fit(X1_train_selected, y_train)
    coef = elastic_net_model.coef_.flatten()
    threshold = 1e-4
    elastic_net_selected_indices = np.where(np.abs(coef) > threshold)[0]
    if len(elastic_net_selected_indices) == 0:
        multistep_selected_features = selected_features
    else:
        multistep_selected_features = selected_features[elastic_net_selected_indices]
    print("\nFinal selected clinical features (ElasticNet):")
    print(feature_names1[multistep_selected_features])
    # Update clinical features with final selection
    X1_train_s = X1_train_s[:, multistep_selected_features]
    X1_test_s = X1_test_s[:, multistep_selected_features]
else:
    print("No clinical features passed univariate logistic regression.")
    multistep_selected_features = np.array([])

# ------------------------------
# Imaging feature processing & selection
# ------------------------------
scaler2 = StandardScaler()
X2_train_s = scaler2.fit_transform(X2_train)
X2_test_s = scaler2.transform(X2_test)

from sklearn.feature_selection import VarianceThreshold, SelectFromModel
var_thresh2 = VarianceThreshold(threshold=0.01)
X2_train_s = var_thresh2.fit_transform(X2_train_s)
X2_test_s = var_thresh2.transform(X2_test_s)

from sklearn.linear_model import ElasticNetCV
elastic_net2 = ElasticNetCV(cv=5, random_state=42)
select_model2 = SelectFromModel(estimator=elastic_net2)
X2_train_s = select_model2.fit_transform(X2_train_s, y_train)
X2_test_s = select_model2.transform(X2_test_s)

# =============================================================================
# 3. Early Feature Fusion
# =============================================================================
X_train_fusion = np.concatenate((X1_train_s, X2_train_s), axis=1)
X_test_fusion = np.concatenate((X1_test_s, X2_test_s), axis=1)

# =============================================================================
# 4. Model Construction and 5-Fold Cross-Validation
# =============================================================================
base_models = [
    ('lr', LogisticRegression(random_state=42, penalty='l2', C=0.1)),
    ('dt', DecisionTreeClassifier(random_state=42, max_depth=5)),
    ('svm', SVC(probability=True, random_state=42, C=0.1))
]

bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42, max_depth=5), 
                                  n_estimators=50, random_state=42)
boosting_model = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=10, random_state=42, max_depth=5), 
                                    n_estimators=50, random_state=42)
stacking_model = StackingClassifier(classifiers=[clf for _, clf in base_models], 
                                    meta_classifier=LogisticRegression(random_state=42, penalty='l2', C=0.1))
rotation_forest_model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=5)
gtb_randomness_model = GradientBoostingClassifier(n_estimators=50, random_state=42, subsample=0.8, max_depth=5)

models = {
    "Bagging": bagging_model,
    "Boosting": boosting_model,
    "Stacking": stacking_model,
    "Rotation Forest": rotation_forest_model,
    "GTB Randomness": gtb_randomness_model
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}
print("\n5-Fold CV AUC on Early Fusion Features:")
for name, model in models.items():
    aucs = cross_val_score(model, X_train_fusion, y_train, cv=kf, scoring='roc_auc', n_jobs=-1)
    cv_results[name] = aucs
    print(f"{name} - Mean AUC: {aucs.mean():.4f} ± {aucs.std():.4f}")

best_model_name = max(cv_results, key=lambda x: cv_results[x].mean())
print(f"\nBest Model: {best_model_name} (Mean AUC: {cv_results[best_model_name].mean():.4f})")

best_model = models[best_model_name]
best_model.fit(X_train_fusion, y_train)
test_auc = roc_auc_score(y_test, best_model.predict_proba(X_test_fusion)[:, 1])
print(f"{best_model_name} Test Set AUC: {test_auc:.4f}")

# =============================================================================
# 5. Plot CV AUC Distribution (Optional)
# =============================================================================
plt.figure(figsize=(10, 6))
for name, aucs in cv_results.items():
    plt.plot(np.arange(1, 6), aucs, marker='o', label=name)
plt.xlabel("Fold")
plt.ylabel("AUC")
plt.title("5-Fold Cross-Validation AUC for Various Models")
plt.legend()
plt.grid()
plt.show()

# =============================================================================
# 6. Show mean AUC for train and test sets in 5-fold
# =============================================================================
print("\n" + "="*40 + " Train/Test Set 5-Fold Cross-Validation Results " + "="*40)

def evaluate_and_plot_roc(models, X_train, y_train, X_test, y_test, kf):
    """Evaluate models on train/test set and plot ROC curves."""
    # Train set
    print("\nTrain set 5-fold CV results:")
    for name, model in models.items():
        train_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='roc_auc')
        print(f"{name} - Mean AUC: {train_scores.mean():.4f} +/- {train_scores.std():.4f}")
    # Test set
    print("\nTest set 5-fold CV results:")
    for name, model in models.items():
        test_scores = cross_val_score(model, X_test, y_test, cv=kf, scoring='roc_auc')
        print(f"{name} - Mean AUC: {test_scores.mean():.4f} +/- {test_scores.std():.4f}")

    # ROC plot
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plot_mean_roc(models, X_train, y_train, kf, "Training Set")
    plt.subplot(1, 2, 2)
    plot_mean_roc(models, X_test, y_test, kf, "Test Set")
    plt.tight_layout()
    plt.savefig('mean_roc_curves_comparison.pdf')
    plt.show()

def plot_mean_roc(models, X, y, cv, title):
    """Plot mean ROC curves."""
    for name, model in models.items():
        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
            if hasattr(model, "predict_proba"):
                model.fit(X[train_idx], y[train_idx])
                y_prob = model.predict_proba(X[val_idx])[:, 1]
            else:
                model.fit(X[train_idx], y[train_idx])
                y_prob = model.decision_function(X[val_idx])
            from sklearn.metrics import roc_curve, auc
            fpr, tpr, _ = roc_curve(y[val_idx], y_prob)
            roc_auc = auc(fpr, tpr)
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(roc_auc)
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr, 
                label=f'{name} (AUC = {mean_auc:.3f} ± {std_auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Mean ROC Curves ({title})')
    plt.legend(loc="lower right", prop={'size': 8})

evaluate_and_plot_roc(models, X_train_fusion, y_train, X_test_fusion, y_test, kf)

print("\n" + "="*40 + " Analysis Complete " + "="*40)

# Optional: save results to Excel
def save_results_to_excel(models, X_train, y_train, X_test, y_test, kf):
    results = []
    for name, model in models.items():
        train_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='roc_auc')
        test_scores = cross_val_score(model, X_test, y_test, cv=kf, scoring='roc_auc')
        results.append({
            'Model': name,
            'Train_AUC_Mean': train_scores.mean(),
            'Train_AUC_Std': train_scores.std(),
            'Test_AUC_Mean': test_scores.mean(),
            'Test_AUC_Std': test_scores.std()
        })
    results_df = pd.DataFrame(results)
    results_df.to_excel('model_comparison_results.xlsx', index=False)
    print("\nDetailed results saved to 'model_comparison_results.xlsx'")

save_results_to_excel(models, X_train_fusion, y_train, X_test_fusion, y_test, kf)

In [6]:
# Late fusion-------------------------------------- Clinical and radiomics modals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, ElasticNetCV
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.base import BaseEstimator, ClassifierMixin, clone
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

# =============================================================================
# 1. Data loading and preprocessing
# =============================================================================
# Load clinical data (modality 1)
clinical_data_path = input("Please enter the clinical data file path: ").strip()
clinical_data = pd.read_excel(clinical_data_path)
X1 = clinical_data.iloc[:, 1:].values
y1 = clinical_data.iloc[:, 0].values
feature_names1 = clinical_data.columns[1:]

# Load imaging data (modality 2)
imaging_data_path = input("Please enter the imaging data file path: ").strip()
imaging_data = pd.read_excel(imaging_data_path)
X2 = imaging_data.iloc[:, 1:].values
y2 = imaging_data.iloc[:, 0].values

# Split data
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, stratify=y1, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, stratify=y2, random_state=42)

# =============================================================================
# 2. Feature processing and selection
# =============================================================================
# Clinical data processing
scaler1 = StandardScaler()
X1_train_s = scaler1.fit_transform(X1_train)
X1_test_s = scaler1.transform(X1_test)

# Imaging data processing
scaler2 = StandardScaler()
X2_train_s = scaler2.fit_transform(X2_train)
X2_test_s = scaler2.transform(X2_test)

# Clinical feature selection - univariate logistic regression
p_values = []
selected_features = []

for i in range(X1_train_s.shape[1]):
    X_single = X1_train_s[:, i].reshape(-1, 1)
    logit_model = sm.Logit(y1_train, sm.add_constant(X_single))
    result = logit_model.fit(disp=0)
    p_value = result.pvalues[1]
    p_values.append(p_value)
    if p_value < 0.05:
        selected_features.append(i)

selected_features = np.array(selected_features)

print("\nInitially selected features (Univariate Logistic Regression):")
print(feature_names1[selected_features])

# Elastic Net for further selection
if len(selected_features) > 0:
    X1_train_selected = X1_train_s[:, selected_features]
    X1_test_selected = X1_test_s[:, selected_features]
    elastic_net_model = LogisticRegression(random_state=42, penalty='elasticnet', 
                                         solver='saga', l1_ratio=0.5, C=1.0, max_iter=5000)
    elastic_net_model.fit(X1_train_selected, y1_train)
    coef = elastic_net_model.coef_.flatten()
    threshold = 1e-4
    elastic_net_selected_indices = np.where(np.abs(coef) > threshold)[0]
    if len(elastic_net_selected_indices) == 0:
        multistep_selected_features = selected_features
    else:
        multistep_selected_features = selected_features[elastic_net_selected_indices]
    print("\nFinal selected features (Elastic Net Regression):")
    print(feature_names1[multistep_selected_features])
    X1_train_s = X1_train_s[:, multistep_selected_features]
    X1_test_s = X1_test_s[:, multistep_selected_features]
else:
    print("No features passed univariate logistic regression selection.")
    multistep_selected_features = np.array([])

# Imaging feature selection
var_thresh2 = VarianceThreshold(threshold=0.01)
X2_train_s = var_thresh2.fit_transform(X2_train_s)
X2_test_s = var_thresh2.transform(X2_test_s)

elastic_net2 = ElasticNetCV(cv=5, random_state=42)
select_model2 = SelectFromModel(estimator=elastic_net2)
X2_train_s = select_model2.fit_transform(X2_train_s, y2_train)
X2_test_s = select_model2.transform(X2_test_s)

# Get features after variance threshold
var_selected_features = np.where(var_thresh2.get_support())[0]
features_after_var = imaging_data.columns[1:][var_selected_features]

# Get Elastic Net selected features
elastic_net_selected_features = np.where(select_model2.get_support())[0]
final_imaging_features = features_after_var[elastic_net_selected_features]

print("\nImaging modality feature selection results:")
print("-" * 80)
print(f"Original feature count: {len(imaging_data.columns[1:])}")
print(f"After variance threshold: {len(features_after_var)}")
print(f"After Elastic Net: {len(final_imaging_features)}")
print("\nFinal selected imaging features:")
for i, feature in enumerate(final_imaging_features, 1):
    print(f"{i}. {feature}")
print("-" * 80)

selected_features_mod2 = final_imaging_features

# =============================================================================
# 3. Define late fusion classifier
# =============================================================================
class LateFusionClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model
        
    def fit(self, X1, X2, y):
        self.base_models_ = [clone(model) for model in self.base_models]
        self.base_models_[0].fit(X1, y)
        self.base_models_[1].fit(X2, y)
        meta_features = self._get_meta_features(X1, X2)
        self.meta_model_ = clone(self.meta_model)
        self.meta_model_.fit(meta_features, y)
        return self
    
    def predict(self, X1, X2):
        meta_features = self._get_meta_features(X1, X2)
        return self.meta_model_.predict(meta_features)
    
    def predict_proba(self, X1, X2):
        meta_features = self._get_meta_features(X1, X2)
        return self.meta_model_.predict_proba(meta_features)
    
    def _get_meta_features(self, X1, X2):
        meta_X1 = self.base_models_[0].predict_proba(X1)[:, 1]
        meta_X2 = self.base_models_[1].predict_proba(X2)[:, 1]
        return np.column_stack((meta_X1, meta_X2))

# =============================================================================
# 4. Evaluate unimodal model performance
# =============================================================================
def evaluate_model_performance(model, X_train, X_test, y_train, y_test, model_name):
    """Evaluate model performance on train and test sets"""
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    train_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='roc_auc')
    model.fit(X_train, y_train)
    y_train_pred = model.predict_proba(X_train)[:, 1]
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_scores = cross_val_score(model, X_test, y_test, cv=kf, scoring='roc_auc')
    y_test_pred = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_test_pred)
    print(f"\n{model_name} model evaluation:")
    print(f"Train CV AUC: {train_scores.mean():.2f} +/- {train_scores.std():.2f}")
    print(f"Train AUC: {train_auc:.2f}")
    print(f"Test CV AUC: {test_scores.mean():.2f} +/- {test_scores.std():.2f}")
    print(f"Test AUC: {test_auc:.2f}")
    return model, train_scores, test_scores, train_auc, test_auc

# Base models
rotation_forest = RandomForestClassifier(
    n_estimators=30,
    max_depth=3,
    min_samples_split=2,
    random_state=42
)
base_rf = RandomForestClassifier(
    n_estimators=10,
    max_depth=3,
    min_samples_split=2,
    random_state=42
)
boosting_model = AdaBoostClassifier(
    estimator=base_rf,
    n_estimators=30,
    random_state=42
)

# Evaluate clinical modality model
clinical_model, clinical_train_scores, clinical_test_scores, clinical_train_auc, clinical_test_auc = \
    evaluate_model_performance(rotation_forest, X1_train_s, X1_test_s, y1_train, y1_test, "Clinical Modality (Rotation Forest)")

# Evaluate imaging modality model
imaging_model, imaging_train_scores, imaging_test_scores, imaging_train_auc, imaging_test_auc = \
    evaluate_model_performance(boosting_model, X2_train_s, X2_test_s, y2_train, y2_test, "Imaging Modality (Boosting)")

# =============================================================================
# 5. Build and evaluate late fusion model
# =============================================================================
kf = KFold(n_splits=5, shuffle=True, random_state=42)
late_fusion_model = LateFusionClassifier(
    base_models=[rotation_forest, boosting_model],
    meta_model=LogisticRegression(random_state=42, penalty='l2', C=0.1)
)

# Evaluate fusion model on training set
train_late_auc_scores = []
for train_idx, val_idx in kf.split(X1_train_s):
    X1_train_fold, X1_val_fold = X1_train_s[train_idx], X1_train_s[val_idx]
    X2_train_fold, X2_val_fold = X2_train_s[train_idx], X2_train_s[val_idx]
    y_train_fold, y_val_fold = y1_train[train_idx], y1_train[val_idx]
    model = clone(late_fusion_model)
    model.fit(X1_train_fold, X2_train_fold, y_train_fold)
    y_prob = model.predict_proba(X1_val_fold, X2_val_fold)[:, 1]
    auc_val = roc_auc_score(y_val_fold, y_prob)
    train_late_auc_scores.append(auc_val)

# Evaluate fusion model on test set
test_late_auc_scores = []
for train_idx, val_idx in kf.split(X1_test_s):
    X1_val_train, X1_val_test = X1_test_s[train_idx], X1_test_s[val_idx]
    X2_val_train, X2_val_test = X2_test_s[train_idx], X2_test_s[val_idx]
    y_val_train, y_val_test = y1_test[train_idx], y1_test[val_idx]
    model = clone(late_fusion_model)
    model.fit(X1_val_train, X2_val_train, y_val_train)
    y_prob = model.predict_proba(X1_val_test, X2_val_test)[:, 1]
    auc_val = roc_auc_score(y_val_test, y_prob)
    test_late_auc_scores.append(auc_val)

print("\nLate fusion model evaluation:")
print("Train CV AUC: {:.2f} +/- {:.2f}".format(
    np.mean(train_late_auc_scores), np.std(train_late_auc_scores)))
print("Test CV AUC: {:.2f} +/- {:.2f}".format(
    np.mean(test_late_auc_scores), np.std(test_late_auc_scores)))

# =============================================================================
# 6. Performance comparison visualization
# =============================================================================
plt.figure(figsize=(10, 8))

# Train final fusion model
late_fusion_model.fit(X1_test_s, X2_test_s, y1_test)

print("\nModel performance summary:")
print("-" * 80)
print("{:<20} {:<25} {:<25}".format("Model", "Training CV AUC", "Testing CV AUC"))
print("-" * 80)
print("{:<20} {:.2f} ± {:.2f} {:<10} {:.2f} ± {:.2f}".format(
    "Clinical Model",
    np.mean(clinical_train_scores), np.std(clinical_train_scores),
    "",
    np.mean(clinical_test_scores), np.std(clinical_test_scores)))
print("{:<20} {:.2f} ± {:.2f} {:<10} {:.2f} ± {:.2f}".format(
    "Imaging Model",
    np.mean(imaging_train_scores), np.std(imaging_train_scores),
    "",
    np.mean(imaging_test_scores), np.std(imaging_test_scores)))
print("{:<20} {:.2f} ± {:.2f} {:<10} {:.2f} ± {:.2f}".format(
    "Fusion Model",
    np.mean(train_late_auc_scores), np.std(train_late_auc_scores),
    "",
    np.mean(test_late_auc_scores), np.std(test_late_auc_scores)))
print("-" * 80)

In [4]:
# Visualization------------------------------------ Clinical feature selection
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import rankdata
import numpy as np
import pandas as pd

def visualize_univariate_logistic(p_values, feature_names, save_path=None):
    """
    Visualize p-values from univariate logistic regression
    """
    mpl.rcParams['font.family'] = 'Arial'
    mpl.rcParams['font.size'] = 16
    plt.figure(figsize=(18, 6))
    
    significance_threshold = 0.05
    sorted_indices = np.argsort(p_values)
    
    # Blue for significant, gray for not significant features
    colors = ['#3498db' if p <= significance_threshold else '#bdc3c7' 
             for p in p_values[sorted_indices]]
    
    bars = plt.bar(range(len(p_values)), -np.log10(p_values[sorted_indices]), color=colors)
    
    plt.axhline(y=-np.log10(significance_threshold), color='red', linestyle='--', 
                label=f'Significance threshold (p={significance_threshold})')
    
    plt.xticks(range(len(p_values)), feature_names[sorted_indices], rotation=45, ha='right', fontsize=16)
    plt.yticks(fontsize=20)
    plt.ylabel('-log10(p-value)', fontsize=20)
    plt.title(' ')
    plt.legend(fontsize=20)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
    plt.show()

def visualize_elastic_net(model, feature_names, save_path=None):
    """
    Visualize the feature coefficients from elastic net regression
    """
    mpl.rcParams['font.family'] = 'Arial'
    mpl.rcParams['font.size'] = 24
    plt.figure(figsize=(8, 6))
    
    coefficients = model.coef_.flatten()
    sorted_indices = np.argsort(np.abs(coefficients))
    
    colors = ['#e74c3c' if coef < 0 else '#3498db' 
             for coef in coefficients[sorted_indices]]
    
    plt.barh(range(len(coefficients)), coefficients[sorted_indices], color=colors)
    plt.yticks(range(len(coefficients)), feature_names[sorted_indices])
    plt.xlabel('Coefficient Value')
    # plt.title('Selected Features Coefficient')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
    plt.show()


# Visualization example usage:
# 1. Visualize univariate logistic regression results
visualize_univariate_logistic(np.array(p_values), feature_names, 'univariate_selection.pdf')

# 2. Visualize elastic net results
visualize_elastic_net(elastic_net_model, feature_names[selected_features], 'elastic_net_selection.pdf')

In [9]:
# Visualization------------------------------------ Feature distribution heat map
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

def create_abbreviated_feature_names(clinical_features, radiomic_features):
    clinical_features = list(clinical_features)
    radiomic_features = list(radiomic_features)
    radiomic_short = [f"RF {i+1}" for i in range(len(radiomic_features))]
    feature_name_abbr = clinical_features + radiomic_short
    abbr_df = pd.DataFrame({
        "Radiomic Abbreviation": radiomic_short,
        "Original Radiomic Feature Name": radiomic_features
    })
    return feature_name_abbr, abbr_df, radiomic_short

def plot_feature_heatmap(X1_train_s, X2_train_s, y1_train, selected_features_mod1, selected_features_mod2):
    """Plot optimized feature distribution heatmap with group color bar on top, 
    clinical features with original names, radiomic features as RF 1,2,..., larger Arial font, group labels Non-response/Response."""
    plt.rcParams["font.sans-serif"] = ["Arial"]
    plt.rcParams["font.family"] = "sans-serif"
    plt.rcParams["axes.unicode_minus"] = False
    plt.rcParams["axes.labelsize"] = 22
    plt.rcParams["xtick.labelsize"] = 20
    plt.rcParams["ytick.labelsize"] = 24
    plt.rcParams["legend.fontsize"] = 22

    all_features, radiomic_df, radiomic_short = create_abbreviated_feature_names(
        selected_features_mod1, selected_features_mod2
    )
    radiomic_df.to_csv("radiomic_feature_abbreviation.csv", index=False)

    df1 = pd.DataFrame(X1_train_s, columns=selected_features_mod1)
    df2 = pd.DataFrame(X2_train_s, columns=radiomic_short)
    df = pd.concat([df1, df2], axis=1)

    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    sort_idx = np.argsort(y1_train)
    df_scaled_sorted = df_scaled.iloc[sort_idx]
    y_sorted = np.array(y1_train)[sort_idx]

    import matplotlib.gridspec as gridspec
    fig = plt.figure(figsize=(28, 12))
    gs = gridspec.GridSpec(2, 1, height_ratios=[0.8, 10], hspace=0.02)
    
    # Group color bar
    ax_label = fig.add_subplot(gs[0])
    n_non = sum(y_sorted == 0)
    n_resp = sum(y_sorted == 1)
    n_samples = len(y_sorted)
    ax_label.axvspan(0, n_non, facecolor='#D6EAF8', alpha=0.5)
    ax_label.axvspan(n_non, n_samples, facecolor='#D5F5E3', alpha=0.5)
    ax_label.text(n_non/2, 0.5, f'Non-response (N={n_non})', 
                  ha='center', va='center', fontsize=28, fontweight='bold', family='Arial')
    ax_label.text(n_non + n_resp/2, 0.5, f'Response (N={n_resp})', 
                  ha='center', va='center', fontsize=28, fontweight='bold', family='Arial')
    ax_label.set_xlim(0, n_samples)
    ax_label.set_ylim(0, 1)
    ax_label.axis('off')
    
    ax1 = fig.add_subplot(gs[1])
    sns.heatmap(
        df_scaled_sorted.T,
        ax=ax1,
        cmap='RdBu_r',
        center=0,
        vmin=-3,
        vmax=3,
        cbar_kws={'label': 'Standardized Value', 'pad': 0.01},
        xticklabels=False,
        yticklabels=True
    )
    ax1.axvline(n_non, color='black', linestyle='--', alpha=0.5)
    ax1.set_yticklabels(df.columns, fontsize=25, family='Arial')
    ax_label.set_position([
        ax1.get_position().x0,
        ax_label.get_position().y0,
        ax1.get_position().width,
        ax_label.get_position().height
    ])
    cbar = ax1.collections[0].colorbar
    cbar.ax.tick_params(labelsize=22)
    cbar.set_label('Standardized Value', fontsize=26, family='Arial')

    plt.tight_layout()
    plt.savefig('feature_heatmap_response_vs_nonresponse.pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plot_pvalue_effectsize_bar(X1_train_s, X2_train_s, y1_train, selected_features_mod1, selected_features_mod2):
    """Plot bar chart of p-values and effect sizes, clinical features original names, radiomic features as RF 1,2,..., larger Arial font."""
    plt.rcParams["font.sans-serif"] = ["Arial"]
    plt.rcParams["font.family"] = "sans-serif"
    plt.rcParams["axes.unicode_minus"] = False
    plt.rcParams["axes.labelsize"] = 22
    plt.rcParams["xtick.labelsize"] = 20
    plt.rcParams["ytick.labelsize"] = 24
    plt.rcParams["legend.fontsize"] = 22

    all_features, radiomic_df, radiomic_short = create_abbreviated_feature_names(
        selected_features_mod1, selected_features_mod2
    )

    df1 = pd.DataFrame(X1_train_s, columns=selected_features_mod1)
    df2 = pd.DataFrame(X2_train_s, columns=radiomic_short)
    df = pd.concat([df1, df2], axis=1)

    p_values = {}
    effect_sizes = {}
    for col in df.columns:
        pos_data = df[col][y1_train == 1]
        neg_data = df[col][y1_train == 0]
        t_stat, t_p = stats.ttest_ind(neg_data, pos_data)
        u_stat, u_p = stats.mannwhitneyu(neg_data, pos_data, alternative='two-sided')
        p_values[col] = {'t_test': t_p, 'mw_test': u_p}
        d = (np.mean(pos_data) - np.mean(neg_data)) / np.sqrt((np.var(pos_data) + np.var(neg_data)) / 2)
        effect_sizes[col] = abs(d)

    fig, ax2 = plt.subplots(figsize=(28, 14))
    p_val_df = pd.DataFrame({
        'Feature': list(p_values.keys()),
        't_test_p': [p_values[k]['t_test'] for k in p_values],
        'mw_test_p': [p_values[k]['mw_test'] for k in p_values],
        'Effect_Size': [effect_sizes[k] for k in p_values]
    })
    p_val_df = p_val_df.sort_values('Effect_Size', ascending=True)
    x = np.arange(len(p_val_df))
    width = 0.35
    ax2.barh(x - width/2, -np.log10(p_val_df['t_test_p']), width, label='t-test', color='skyblue')
    ax2.barh(x + width/2, -np.log10(p_val_df['mw_test_p']), width, label='Mann-Whitney U test', color='lightcoral')
    for i, (idx, row) in enumerate(p_val_df.iterrows()):
        ax2.text(0.1, i, f"d={row['Effect_Size']:.2f}", va='center', fontsize=28, family='Arial')
    ax2.set_yticks(x)
    ax2.set_yticklabels(p_val_df['Feature'], fontsize=32, family='Arial')
    significance_levels = [0.05, 0.01, 0.001]
    for level in significance_levels:
        ax2.axvline(-np.log10(level), color='gray', linestyle='--', alpha=0.5)
        ax2.text(-np.log10(level), len(p_val_df), f'p={level}', ha='center', va='bottom', fontsize=28, family='Arial')
    ax2.set_xlabel('-log10(p-value)', fontsize=28, family='Arial')
    ax2.legend(fontsize=28)
    ax2.tick_params(axis='x', labelsize=28)
    plt.tight_layout()
    plt.savefig('feature_statistical_significance_bar.pdf', dpi=300, bbox_inches='tight')
    plt.show()

# Example usage:
plot_feature_heatmap(
    X1_train_s, 
    X2_train_s, 
    y1_train, 
    feature_names1[multistep_selected_features],   # Clinical features
    final_imaging_features                        # Radiomic features
)
plot_pvalue_effectsize_bar(
    X1_train_s, 
    X2_train_s, 
    y1_train, 
    feature_names1[multistep_selected_features], 
    final_imaging_features
)

In [10]:
# Visualization------------------------------------ P value difference map
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# Global font settings
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 16

def plot_pvalue_bars(
    X1_train_s, X2_train_s, y1_train,
    selected_features_mod1, selected_features_mod2,
    save_path="feature_pvalue.pdf"
):
    """
    Plot bar charts of p-values and effect sizes for features and save as PDF.
    """
    df1 = pd.DataFrame(X1_train_s, columns=selected_features_mod1)
    df2 = pd.DataFrame(X2_train_s, columns=selected_features_mod2)
    df = pd.concat([df1, df2], axis=1)

    p_values = {}
    effect_sizes = {}
    for col in df.columns:
        pos_data = df[col][y1_train == 1]
        neg_data = df[col][y1_train == 0]
        t_stat, t_p = stats.ttest_ind(neg_data, pos_data)
        u_stat, u_p = stats.mannwhitneyu(neg_data, pos_data, alternative='two-sided')
        p_values[col] = {'t_test': t_p, 'mw_test': u_p}
        d = (np.mean(pos_data) - np.mean(neg_data)) / np.sqrt((np.var(pos_data) + np.var(neg_data)) / 2)
        effect_sizes[col] = abs(d)

    p_val_df = pd.DataFrame({
        'Feature': list(p_values.keys()),
        't_test_p': [p_values[k]['t_test'] for k in p_values],
        'mw_test_p': [p_values[k]['mw_test'] for k in p_values],
        'Effect_Size': [effect_sizes[k] for k in p_values]
    })
    p_val_df = p_val_df.sort_values('Effect_Size', ascending=True)

    x = np.arange(len(p_val_df))
    width = 0.35
    fig, ax2 = plt.subplots(figsize=(12, 12))
    bars1 = ax2.barh(
        x - width/2, -np.log10(p_val_df['t_test_p']),
        width, label='t-test', color='skyblue'
    )
    bars2 = ax2.barh(
        x + width/2, -np.log10(p_val_df['mw_test_p']),
        width, label='Mann-Whitney U test', color='lightcoral'
    )

    for i, (idx, row) in enumerate(p_val_df.iterrows()):
        ax2.text(0.1, i, f"d={row['Effect_Size']:.2f}", 
                 va='center', fontsize=16, fontname='Arial')

    ax2.set_yticks(x)
    ax2.set_yticklabels(p_val_df['Feature'], fontsize=16, fontname='Arial')
    significance_levels = [0.05, 0.01, 0.001]
    for level in significance_levels:
        ax2.axvline(-np.log10(level), color='gray', linestyle='--', alpha=0.5)
        ax2.text(-np.log10(level), len(p_val_df), f'p={level}', 
                 ha='center', va='bottom', fontsize=16, fontname='Arial')
    plt.title('Statistical Significance and Effect Sizes', pad=20, fontsize=16, fontname='Arial')
    plt.xlabel('-log10(p-value)', fontsize=16, fontname='Arial')
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

# Example usage (replace with your actual variable names)
plot_pvalue_bars(
    X1_train_s, X2_train_s, y1_train,
    feature_names1[multistep_selected_features],
    final_imaging_features,
    save_path='feature_pvalue.pdf'
)

In [11]:
# Visualization------------------------------------ SHAP for fusion model
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import LinearSegmentedColormap
from scipy.spatial import KDTree

# Assume variables are already defined:
# X1_test_s, X2_test_s, late_fusion_model, feature_names1, multistep_selected_features, final_imaging_feature_names

# Combine clinical and imaging test data into one array (fusion input)
d1 = X1_test_s.shape[1]
d2 = X2_test_s.shape[1]
X_fusion = np.concatenate((X1_test_s, X2_test_s), axis=1)

# Abbreviate imaging feature names (RF 1...), keep clinical feature names
clinical_feature_names = list(feature_names1[multistep_selected_features])
imaging_feature_names = list(final_imaging_feature_names)
imaging_feature_names_abbr = [f"RF {i+1}" for i in range(len(imaging_feature_names))]
fusion_feature_names = clinical_feature_names + imaging_feature_names_abbr

# Save abbreviation table
pd.DataFrame({
    "RF Abbreviation": imaging_feature_names_abbr,
    "Original Imaging Feature Name": imaging_feature_names
}).to_csv("fusion_shap_rf_abbreviation.csv", index=False)

print("Fusion input shape:", X_fusion.shape)
print("Fusion feature names:", fusion_feature_names)

# Define a wrapper function for the fusion model.
def fusion_model_wrapper(X):
    X1 = X[:, :d1]
    X2 = X[:, d1:]
    return late_fusion_model.predict_proba(X1, X2)

# Create a SHAP explainer for the fusion model.
explainer_fusion = shap.Explainer(fusion_model_wrapper, X_fusion)

# Compute SHAP values.
shap_values_obj = explainer_fusion(X_fusion)
shap_values_positive = shap_values_obj.values[:, :, 1]
print("SHAP values shape for positive class:", shap_values_positive.shape)

# Construct DataFrames for the SHAP values and the fusion features.
shap_df_fusion = pd.DataFrame(shap_values_positive, columns=fusion_feature_names)
X_fusion_df = pd.DataFrame(X_fusion, columns=fusion_feature_names)
print("SHAP DataFrame shape:", shap_df_fusion.shape)

# --- SHAP Scatter Plot Visualization ---
jitter_scale = 0.15
distance_threshold = 0.05
custom_cmap = LinearSegmentedColormap.from_list("custom", ["#3465A4", "#E6E6E6", "#CC3333"])

# Compute mean absolute SHAP values and sort features
mean_abs_shap_values = shap_df_fusion.abs().mean().sort_values(ascending=False)
sorted_features = mean_abs_shap_values.index

# Prepare plotting data
plot_data = []
for feature in sorted_features:
    for shap_val, feature_val in zip(shap_df_fusion[feature], X_fusion_df[feature]):
        normalized_value = (feature_val - X_fusion_df[feature].min()) / (
            X_fusion_df[feature].max() - X_fusion_df[feature].min()
        )
        plot_data.append({
            "Feature": feature, 
            "SHAP Value": shap_val, 
            "Normalized Value": normalized_value
        })
plot_df = pd.DataFrame(plot_data)

# Create a KDTree for the SHAP values
all_points = plot_df["SHAP Value"].values.reshape(-1, 1)
tree = KDTree(all_points)

# --- Define feature groups and associated colors ---
n_clinical = len(clinical_feature_names)
feature_groups = {
    "Clinical": fusion_feature_names[: n_clinical],
    "Imaging": fusion_feature_names[n_clinical:]
}
group_colors = {
    "Clinical": "#ABDAEC",
    "Imaging": "#6A9ACF",
}
feature_colors = []
for feature in sorted_features:
    assigned_color = "#cccccc"
    for group, features in feature_groups.items():
        if feature in features:
            assigned_color = group_colors.get(group, "#000000")
            break
    feature_colors.append(assigned_color)

# --- Set up figure and grid layout ---
plt.rcParams["font.sans-serif"] = ["Arial"]
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["axes.unicode_minus"] = False

fig = plt.figure(figsize=(15, 14))
gs = gridspec.GridSpec(2, 2, width_ratios=[0.85, 0.15], height_ratios=[0.05, 0.85])
ax = plt.subplot(gs[1, 0])

# Plot scatter points for each feature with jitter
for i, feature in enumerate(sorted_features):
    subset = plot_df[plot_df["Feature"] == feature]
    shap_values = subset["SHAP Value"].values
    jitter = np.zeros(len(shap_values))
    for idx, shap_value in enumerate(shap_values):
        neighbors = tree.query_ball_point([shap_value], r=distance_threshold)
        if len(neighbors) > 1:
            jitter[idx] = np.random.normal(loc=0, scale=jitter_scale)
    ax.scatter(
        subset["SHAP Value"], 
        jitter + i,  # y position plus jitter
        c=subset["Normalized Value"],  
        cmap=custom_cmap,  
        s=28, 
        alpha=0.7,
    )

# Add a vertical reference line at SHAP value 0.
ax.axvline(x=0, color="gray", linestyle="--", linewidth=1.5, alpha=0.5)
ax.set_yticks(range(len(sorted_features)))
ax.set_yticklabels(sorted_features, fontsize=24, family="Arial")
ax.tick_params(axis='x', labelsize=24)
for label in ax.get_xticklabels():
    label.set_fontname('Arial')
    label.set_fontsize(24)
ax.invert_yaxis()
ax.grid(False)

# --- Create a horizontal bar chart showing mean(|SHAP value|) per feature ---
ax_bar = plt.subplot(gs[1, 1], sharey=ax)
bars = ax_bar.barh(
    range(len(sorted_features)),
    mean_abs_shap_values.values,
    color=feature_colors,
    alpha=0.7,
)
ax_bar.set_xlim(0, mean_abs_shap_values.max() * 2.0)
ax_bar.set_xticks([])
ax_bar.set_xlabel("|SHAP value|", fontsize=22, family="Arial", labelpad=15)
for bar, value in zip(bars, mean_abs_shap_values.values):
    ax_bar.text(
        value + mean_abs_shap_values.max() * 0.02,
        bar.get_y() + bar.get_height() / 2,
        f"{value:.3f}",
        va="center",
        fontsize=24,
        color="black",
        family="Arial"
    )
ax_bar.tick_params(axis="y", which="both", left=False, right=False, labelleft=False)
ax_bar.grid(False)

# --- Add a colorbar for the scatter plot ---
cax = plt.subplot(gs[0, 0])
sm = plt.cm.ScalarMappable(cmap=custom_cmap)
sm.set_array([])
cbar = fig.colorbar(sm, cax=cax, orientation="horizontal")
cbar.outline.set_visible(False)
cbar.ax.tick_params(labelsize=12, labelcolor="black")
cbar.ax.xaxis.set_ticks([])
cbar.ax.text(-0.02, 0.5, "Low", ha="center", va="center", transform=cbar.ax.transAxes, fontsize=24, family="Arial")
cbar.ax.text(1.02, 0.5, "High", ha="center", va="center", transform=cbar.ax.transAxes, fontsize=24, family="Arial")

plt.subplots_adjust(top=0.8, wspace=0.05)
plt.savefig("fusion_shap for response.pdf", format="pdf", bbox_inches="tight", dpi=1200)
plt.show()

print("Fusion SHAP visualization finished.")

In [12]:
# Visualization------------------------------------ DCA curve
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import clone

def calculate_net_benefit(y_true, y_pred_prob, threshold):
    """Calculate net benefit at a specific threshold."""
    y_pred = (y_pred_prob >= threshold).astype(int)
    TP = np.sum((y_pred == 1) & (y_true == 1))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    n = len(y_true)
    
    if TP + FP == 0:
        return 0
    
    net_benefit = (TP/n) - (FP/n) * (threshold/(1-threshold))
    return net_benefit

def plot_dca_curves(X1_train, X2_train, y_train, X1_test, X2_test, y_test, 
                   clinical_model, imaging_model, late_fusion_model, dataset_type="Test"):
    """Plot optimized DCA curves (no title)."""
    thresholds = np.arange(0.01, 1.0, 0.01)
    
    if dataset_type == "Train":
        X1, X2, y = X1_train, X2_train, y_train
    else:
        X1, X2, y = X1_test, X2_test, y_test
    
    # Fit models and predict probabilities
    clinical_model_fitted = clone(clinical_model).fit(X1, y)
    imaging_model_fitted = clone(imaging_model).fit(X2, y)
    fusion_model_fitted = clone(late_fusion_model)
    fusion_model_fitted.fit(X1, X2, y)
    
    y_pred_clinical = clinical_model_fitted.predict_proba(X1)[:, 1]
    y_pred_imaging = imaging_model_fitted.predict_proba(X2)[:, 1]
    y_pred_fusion = fusion_model_fitted.predict_proba(X1, X2)[:, 1]
    
    # Calculate net benefits
    nb_clinical = [calculate_net_benefit(y, y_pred_clinical, threshold) for threshold in thresholds]
    nb_imaging = [calculate_net_benefit(y, y_pred_imaging, threshold) for threshold in thresholds]
    nb_fusion = [calculate_net_benefit(y, y_pred_fusion, threshold) for threshold in thresholds]
    nb_all = [calculate_net_benefit(y, np.ones_like(y), threshold) for threshold in thresholds]
    nb_none = np.zeros_like(thresholds)
    
    # Optimized plotting
    plt.figure(figsize=(7, 7))
    
    # Elegant color scheme
    plt.plot(thresholds, nb_clinical, '--', color='#4B88B5', linewidth=2, label='Clinical Model')
    plt.plot(thresholds, nb_imaging, '--', color='#50A35D', linewidth=2, label='Imaging Model')
    plt.plot(thresholds, nb_fusion, '-', color='#C44E52', linewidth=2, label='Fusion Model')
    plt.plot(thresholds, nb_all, '--', color='#808080', linewidth=1.5, label='Treat All')
    plt.plot(thresholds, nb_none, ':', color='#808080', linewidth=1.5, label='Treat None')
    
    plt.xlabel('Threshold Probability', fontsize=22)
    plt.ylabel('Net Benefit', fontsize=22)
    # No title as requested
    
    plt.legend(loc='lower left', frameon=False, fontsize=20)
    plt.grid(False)
    plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5, alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', linewidth=0.5, alpha=0.3)
    plt.xlim(0, 1)
    plt.ylim(min(min(nb_clinical), min(nb_imaging), min(nb_fusion))-0.05,
             max(max(nb_clinical), max(nb_imaging), max(nb_fusion))+0.05)
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)
    plt.savefig(f'dca_curves_{dataset_type.lower()}.pdf', bbox_inches='tight', dpi=300)
    plt.show()

# Plot DCA curves for training set
plot_dca_curves(X1_train_s, X2_train_s, y1_train,
                X1_test_s, X2_test_s, y1_test,
                clinical_model, imaging_model, late_fusion_model, 
                dataset_type="Train")

# Plot DCA curves for test set
plot_dca_curves(X1_train_s, X2_train_s, y1_train,
                X1_test_s, X2_test_s, y1_test,
                clinical_model, imaging_model, late_fusion_model, 
                dataset_type="Test")

In [13]:
# Visualization------------------------------------ Calibration curve
from sklearn.calibration import calibration_curve
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np

def plot_calibration_curves(y_train, y_test, probs_train, probs_test, model_names):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Calibration plot for training set
    ax1.plot([0, 1], [0, 1], 'k:', label='Perfectly calibrated')
    for name, probs in zip(model_names, probs_train):
        prob_true, prob_pred = calibration_curve(y_train, probs, n_bins=5)
        ax1.plot(prob_pred, prob_true, marker='o', label=name)
    
    ax1.set_xlabel('Mean predicted probability')
    ax1.set_ylabel('Fraction of positives')
    ax1.set_title('Calibration Plot (Training Set)')
    ax1.legend(loc='lower right')
    
    # Set grid for training set
    ax1.grid(True, alpha=0.3)
    ax1.grid(True, which='minor', alpha=0.1)
    ax1.minorticks_on()
    
    # Calibration plot for test set
    ax2.plot([0, 1], [0, 1], 'k:', label='Perfectly calibrated')
    for name, probs in zip(model_names, probs_test):
        prob_true, prob_pred = calibration_curve(y_test, probs, n_bins=5)
        ax2.plot(prob_pred, prob_true, marker='o', label=name)
    
    ax2.set_xlabel('Mean predicted probability')
    ax2.set_ylabel('Fraction of positives')
    ax2.set_title('Calibration Plot (Test Set)')
    ax2.legend(loc='lower right')
    
    # Set grid for test set
    ax2.grid(True, alpha=0.1)
    ax2.grid(True, which='minor', alpha=0)
    ax2.minorticks_on()
    
    plt.tight_layout()
    plt.savefig('calibration_curves.pdf', dpi=300, bbox_inches='tight')
    plt.show()

# Prepare training set predicted probabilities
clinical_probs_train = clinical_model.predict_proba(X1_train_s)[:, 1]
imaging_probs_train = imaging_model.predict_proba(X2_train_s)[:, 1]

# Prepare fusion features
X_fusion_train = np.hstack((X1_train_s, X2_train_s))
X_fusion_test = np.hstack((X1_test_s, X2_test_s))

# Train a fusion model using RandomForest for calibration
fusion_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
fusion_classifier.fit(X_fusion_train, y1_train)

# Get predicted probabilities for fusion model
fusion_probs_train = fusion_classifier.predict_proba(X_fusion_train)[:, 1]
fusion_probs_test = fusion_classifier.predict_proba(X_fusion_test)[:, 1]

# Prepare test set predicted probabilities
clinical_probs_test = clinical_model.predict_proba(X1_test_s)[:, 1]
imaging_probs_test = imaging_model.predict_proba(X2_test_s)[:, 1]

# Model names
model_names = ['Clinical Model', 'Imaging Model', 'Fusion Model']

# Probabilities for train and test
probs_train = [clinical_probs_train, imaging_probs_train, fusion_probs_train]
probs_test = [clinical_probs_test, imaging_probs_test, fusion_probs_test]

# Plot calibration curves
plot_calibration_curves(y1_train, y1_test, probs_train, probs_test, model_names)

In [15]:
# Visualization------------------------------------ Model selection performance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 数据部分，'CR'改为'Fusion'
data = {
    'Model_Type': ['Radiomics']*5 + ['Clinical']*5 + ['Early Fusion']*5,
    'Algorithm': ['Bagging', 'Boosting', 'Stacking', 'RF', 'GTBR']*3,
    'Training': [0.76, 0.78, 0.76, 0.78, 0.72,
                0.76, 0.76, 0.79, 0.74, 0.74,
                0.81, 0.84, 0.75, 0.84, 0.79],
    'Training_CI_Lower': [0.67, 0.68, 0.64, 0.66, 0.61,
                         0.71, 0.70, 0.71, 0.68, 0.69,
                         0.69, 0.72, 0.68, 0.74, 0.70],
    'Training_CI_Upper': [0.85, 0.88, 0.88, 0.90, 0.83,
                         0.81, 0.82, 0.87, 0.80, 0.79,
                         0.92, 0.96, 0.82, 0.95, 0.88],
    'Testing': [0.80, 0.86, 0.78, 0.85, 0.87,
                0.73, 0.75, 0.73, 0.76, 0.55,
                0.83, 0.86, 0.78, 0.87, 0.83],
    'Testing_CI_Lower': [0.62, 0.74, 0.66, 0.72, 0.73,
                        0.64, 0.56, 0.60, 0.64, 0.44,
                        0.68, 0.74, 0.70, 0.75, 0.69],
    'Testing_CI_Upper': [0.98, 0.98, 0.90, 0.98, 1.00,
                        0.82, 0.94, 0.86, 0.88, 0.66,
                        0.98, 0.98, 0.85, 0.98, 0.97]
}

df = pd.DataFrame(data)

# 设置全局字体为Arial，字体大小同前
plt.rcParams["font.sans-serif"] = ["Arial"]
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 20
plt.rcParams["ytick.labelsize"] = 20
plt.rcParams["legend.fontsize"] = 24
plt.rcParams["figure.titlesize"] = 22

plt.figure(figsize=(25, 8))
ax = plt.gca()

sns.set_style("white")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

colors = ['#8ecae6', '#219ebc']
n_algorithms = 5
width = 0.40
group_spacing = 0.8

model_types = ['Radiomics', 'Clinical', 'Early Fusion']

for i, model_type in enumerate(model_types):
    model_data = df[df['Model_Type'] == model_type]
    x = np.arange(n_algorithms) + i * (n_algorithms + group_spacing)
    # 训练集
    train_bars = ax.bar(x - width/2, model_data['Training'], width, 
                       label='Training' if i == 0 else "", color=colors[0])
    # 测试集
    test_bars = ax.bar(x + width/2, model_data['Testing'], width,
                      label='Test' if i == 0 else "", color=colors[1])
    # 训练误差棒
    ax.errorbar(x - width/2, model_data['Training'],
           yerr=[model_data['Training'] - model_data['Training_CI_Lower'],
                 model_data['Training_CI_Upper'] - model_data['Training']],
           fmt='none', color='black', capsize=5, alpha=0.2,
           elinewidth=0.8, capthick=0.8)
    # 测试误差棒
    ax.errorbar(x + width/2, model_data['Testing'],
           yerr=[model_data['Testing'] - model_data['Testing_CI_Lower'],
                 model_data['Testing_CI_Upper'] - model_data['Testing']],
           fmt='none', color='black', capsize=5, alpha=0.2,
           elinewidth=0.8, capthick=0.8)
    # 添加数值标签
    for bars in [train_bars, test_bars]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=20, fontfamily="Arial")
    # 上方模型类型标签
    plt.text((i * (n_algorithms + group_spacing) + 2), 1.08, model_type,
             ha='center', va='bottom', fontsize=28, fontweight='bold', fontfamily="Arial")
    # 垂直分隔虚线
    if i < 2:
        separation_x = x[-1] + width + group_spacing/2
        plt.axvline(x=separation_x, color='gray', linestyle='--', alpha=0.3)

# 设置x轴标签
all_x = np.array([np.arange(n_algorithms) + i * (n_algorithms + group_spacing) for i in range(3)]).flatten()
plt.xticks(all_x, ['Bagging', 'Boosting', 'Stacking', 'RF', 'GTBR'] * 3, rotation=0, fontfamily="Arial", fontsize=20)

# plt.xlabel('Algorithm', fontsize=18, labelpad=10, fontfamily="Arial")
plt.ylabel('AUC', fontsize=20, fontfamily="Arial")
# plt.title('AUC Comparison Across Different Models',
#           fontsize=22, pad=20, fontfamily="Arial")
plt.legend(loc='upper right', frameon=False)

plt.ylim(0, 1.12)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.savefig("auc_multimodel.pdf", format="pdf", bbox_inches="tight", dpi=300)
plt.show()

In [16]:
# Visualization------------------------------------ Final AUC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data
data = {
    'Model': ['Clinical Model', 'Radiomics Model', 'LFICKI-TR'],
    'Training': [0.76, 0.76, 0.82],
    'Training_CI_Lower': [0.71, 0.66, 0.72],
    'Training_CI_Upper': [0.81, 0.86, 0.92],
    'Testing': [0.73, 0.88, 0.90],
    'Testing_CI_Lower': [0.50, 0.78, 0.80],
    'Testing_CI_Upper': [0.96, 0.98, 1.00],
    'External': [0.70, 0.84, 0.85]
}
df = pd.DataFrame(data)

# Set global font to Arial
plt.rcParams["font.sans-serif"] = ["Arial"]
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["xtick.labelsize"] = 18
plt.rcParams["ytick.labelsize"] = 22
plt.rcParams["legend.fontsize"] = 22
plt.rcParams["figure.titlesize"] = 22

plt.figure(figsize=(12, 9))
ax = plt.gca()

# Set style
sns.set_style("white")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Color-blind friendly, high contrast
colors = ['#8ecae6', '#219ebc', '#126782']

width = 0.25
x = np.arange(len(data['Model']))

# Bar plots
train_bars = ax.bar(x - width, df['Training'], width, label='Training', color=colors[0])
test_bars = ax.bar(x, df['Testing'], width, label='Testing', color=colors[1])
external_bars = ax.bar(x + width, df['External'], width, label='External Validation', color=colors[2])

# Error bars
ax.errorbar(x - width, df['Training'],
           yerr=[df['Training'] - df['Training_CI_Lower'],
                 df['Training_CI_Upper'] - df['Training']],
           fmt='none', color='black', capsize=5, alpha=0.3,
           elinewidth=1.5, capthick=1.5)

ax.errorbar(x, df['Testing'],
           yerr=[df['Testing'] - df['Testing_CI_Lower'],
                 df['Testing_CI_Upper'] - df['Testing']],
           fmt='none', color='black', capsize=5, alpha=0.3,
           elinewidth=1.5, capthick=1.5)

# Value labels
def add_value_labels(bars, fontsize=24):
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.2f}', ha='center', va='bottom', fontsize=fontsize, fontfamily="Arial")

add_value_labels(train_bars)
add_value_labels(test_bars)
add_value_labels(external_bars)

# X-axis labels
plt.xticks(x, data['Model'], fontsize=24, fontfamily="Arial")

plt.ylabel('AUC', fontsize=24, fontfamily="Arial")
plt.legend(loc='upper left', frameon=False)

ymin = 0.2
max_heights_training = df['Training'] + (df['Training_CI_Upper'] - df['Training'])
max_heights_testing = df['Testing'] + (df['Testing_CI_Upper'] - df['Testing'])
ymax = max(max_heights_training.max(), max_heights_testing.max(), df['External'].max()) + 0.1
plt.ylim(ymin, ymax)

plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.savefig("auc_comparison.pdf", format="pdf", bbox_inches="tight", dpi=300)
plt.show()

In [17]:
# Visualization------------------------------------ Feature correlation matrix 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

def create_abbreviated_feature_names(clinical_features, radiomic_features):
    """Generate RF 1,2,... abbreviations for radiomic features, and return mapping DataFrame."""
    clinical_features = list(clinical_features)
    radiomic_features = list(radiomic_features)
    radiomic_short = [f"RF {i+1}" for i in range(len(radiomic_features))]
    feature_name_abbr = clinical_features + radiomic_short
    abbr_df = pd.DataFrame({
        "Radiomic Abbreviation": radiomic_short,
        "Original Radiomic Feature Name": radiomic_features
    })
    return feature_name_abbr, abbr_df, radiomic_short

def plot_feature_correlation_matrix(X1_train_s, X2_train_s, selected_features_mod1, selected_features_mod2):
    """
    Plot feature correlation matrix (radiomic features auto-abbreviated as RF, Arial font, 
    bigger font size, only last row and first column have labels).
    """
    # Set global font
    plt.rcParams["font.sans-serif"] = ["Arial"]
    plt.rcParams["font.family"] = "sans-serif"
    plt.rcParams["axes.unicode_minus"] = False

    # Feature name abbreviation
    all_features, radiomic_df, radiomic_short = create_abbreviated_feature_names(
        list(selected_features_mod1), list(selected_features_mod2)
    )
    # Save mapping
    radiomic_df.to_csv("radiomic_feature_abbreviation.csv", index=False)

    # Merge data and feature names
    df = pd.DataFrame(np.hstack([X1_train_s, X2_train_s]),
                     columns=all_features)
    n = len(df.columns)
    
    # Compute correlation coefficient matrix and p-values
    corr = df.corr()
    p_values = pd.DataFrame(np.zeros((n, n)), columns=df.columns, index=df.columns)
    for i in range(n):
        for j in range(n):
            if i != j:
                _, p = pearsonr(df.iloc[:, i], df.iloc[:, j])
                p_values.iloc[i, j] = p

    # Create figure
    fig, axes = plt.subplots(n, n, figsize=(2.3 * n, 2.3 * n))

    for i in range(n):
        for j in range(n):
            ax = axes[i, j]
            if i == j:
                # Diagonal: plot histogram
                sns.histplot(df.iloc[:, i], kde=True, ax=ax)
                ax.set_title(df.columns[i], fontsize=18, pad=6, family="Arial")
            elif i > j:
                # Lower triangle: scatter with fit line and confidence interval
                sns.scatterplot(x=df.iloc[:, j], y=df.iloc[:, i], ax=ax, edgecolor=None)
                X = sm.add_constant(df.iloc[:, j])
                model = sm.OLS(df.iloc[:, i], X).fit()
                x_values = np.linspace(df.iloc[:, j].min(), df.iloc[:, j].max(), 100)
                X_pred = sm.add_constant(x_values)
                y_pred = model.predict(X_pred)
                conf_int_pred = model.get_prediction(X_pred).conf_int()
                ax.plot(x_values, y_pred, color="red")
                ax.fill_between(x_values, conf_int_pred[:, 0], conf_int_pred[:, 1], 
                              color="blue", alpha=0.2)
            else:
                # Upper triangle: heatmap with significance annotation
                sns.heatmap(pd.DataFrame([[corr.iloc[i, j]]]), 
                          cmap=sns.diverging_palette(240, 10, as_cmap=True),
                          cbar=False, annot=True, fmt=".2f", square=True, 
                          ax=ax, vmin=-1, vmax=1,
                          annot_kws={"size": 22, "family": "Arial"})
                # Significance annotation
                p_val = p_values.iloc[i, j]
                if p_val < 0.001:
                    ax.text(0.5, 0.8, "***", color="black", ha="center", 
                           va="center", transform=ax.transAxes, fontsize=16, family="Arial")
                elif p_val < 0.01:
                    ax.text(0.5, 0.8, "**", color="black", ha="center", 
                           va="center", transform=ax.transAxes, fontsize=16, family="Arial")
                elif p_val < 0.05:
                    ax.text(0.5, 0.8, "*", color="black", ha="center", 
                           va="center", transform=ax.transAxes, fontsize=16, family="Arial")

            # Axis labels: only keep for last row and first column
            if i < n - 1:
                ax.set_xticks([])
                ax.set_xticklabels([])
            else:
                ax.set_xticks([0.5])
                ax.set_xticklabels([df.columns[j]], fontsize=16, family="Arial", rotation=60, ha="right")
            if j > 0:
                ax.set_yticks([])
                ax.set_yticklabels([])
            else:
                ax.set_yticks([0.5])
                ax.set_yticklabels([df.columns[i]], fontsize=16, family="Arial", rotation=0, va="center")

            # Diagonal: adjust title position
            if i == j:
                ax.set_title(df.columns[i], fontsize=18, pad=6, family="Arial")

    # Adjust layout
    plt.subplots_adjust(hspace=0.4, wspace=0.4)

    # Add colorbar
    fig.subplots_adjust(right=0.86)
    cbar_ax = fig.add_axes([0.88, 0.15, 0.03, 0.7])
    norm = plt.Normalize(vmin=-1, vmax=1)
    sm_map = plt.cm.ScalarMappable(cmap=sns.diverging_palette(240, 10, as_cmap=True), 
                                  norm=norm)
    sm_map.set_array([])
    cbar = fig.colorbar(sm_map, cax=cbar_ax)
    cbar.ax.tick_params(labelsize=18)
    cbar.set_label("Pearson correlation", fontsize=20, family="Arial")

    plt.savefig("Feature_Correlation_Matrix.pdf", format='pdf', bbox_inches='tight', dpi=300)
    plt.show()

# Example usage
print("\nPlotting feature correlation matrix...")
plot_feature_correlation_matrix(
    X1_train_s,
    X2_train_s,
    list(feature_names1[multistep_selected_features]),
    list(final_imaging_features)
)

In [ ]:
# External validation
# =============================================================================
# 7. External Validation Set Evaluation
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score

try:
    # Load external validation clinical data
    clinical_val_data_path = input("Please enter the external clinical validation file path: ").strip()
    clinical_val_data = pd.read_excel(clinical_val_data_path)
    X1_val = clinical_val_data.iloc[:, 1:].values
    y1_val = clinical_val_data.iloc[:, 0].values

    # Load external validation imaging data
    imaging_val_data_path = input("Please enter the external imaging validation file path: ").strip()
    imaging_val_data = pd.read_excel(imaging_val_data_path)
    X2_val = imaging_val_data.iloc[:, 1:].values
    y2_val = imaging_val_data.iloc[:, 0].values

    # =============================================================================
    # 7.1 Clinical data preprocessing and feature selection
    # =============================================================================
    # Standardization
    X1_val_s = scaler1.transform(X1_val)
    
    # Apply previous feature selection
    if len(multistep_selected_features) > 0:
        X1_val_s = X1_val_s[:, multistep_selected_features]
        print("\nClinical features used for external validation:")
        print(feature_names1[multistep_selected_features])
    
    # =============================================================================
    # 7.2 Imaging data preprocessing and feature selection
    # =============================================================================
    # Standardization
    X2_val_s = scaler2.transform(X2_val)
    
    # Apply previous feature selection
    X2_val_s = var_thresh2.transform(X2_val_s)
    X2_val_s = select_model2.transform(X2_val_s)

    # =============================================================================
    # 7.3 Evaluate unimodal models on external validation set
    # =============================================================================
    # Clinical model evaluation
    clinical_val_pred = clinical_model.predict_proba(X1_val_s)[:, 1]
    clinical_val_auc = roc_auc_score(y1_val, clinical_val_pred)
    fpr_clinical_val, tpr_clinical_val, _ = roc_curve(y1_val, clinical_val_pred)
    
    # Imaging model evaluation
    imaging_val_pred = imaging_model.predict_proba(X2_val_s)[:, 1]
    imaging_val_auc = roc_auc_score(y2_val, imaging_val_pred)
    fpr_imaging_val, tpr_imaging_val, _ = roc_curve(y2_val, imaging_val_pred)
    
    # Fusion model evaluation
    fusion_val_pred = late_fusion_model.predict_proba(X1_val_s, X2_val_s)[:, 1]
    fusion_val_auc = roc_auc_score(y1_val, fusion_val_pred)
    fpr_fusion_val, tpr_fusion_val, _ = roc_curve(y1_val, fusion_val_pred)

    # =============================================================================
    # 7.4 Output external validation results
    # =============================================================================
    print("\nExternal validation performance results:")
    print("-" * 80)
    print("{:<20} {:<25}".format("Model", "External Validation AUC"))
    print("-" * 80)
    print("{:<20} {:.4f}".format("Clinical Model", clinical_val_auc))
    print("{:<20} {:.4f}".format("Imaging Model", imaging_val_auc))
    print("{:<20} {:.4f}".format("Fusion Model", fusion_val_auc))
    print("-" * 80)

    # =============================================================================
    # 7.5 Plot ROC curves for external validation set
    # =============================================================================
    plt.figure(figsize=(10, 8))
    
    plt.plot(fpr_clinical_val, tpr_clinical_val, 
             label=f'Clinical Model (AUC = {clinical_val_auc:.4f})', 
             linestyle='--')
    plt.plot(fpr_imaging_val, tpr_imaging_val, 
             label=f'Imaging Model (AUC = {imaging_val_auc:.4f})', 
             linestyle=':')
    plt.plot(fpr_fusion_val, tpr_fusion_val, 
             label=f'Fusion Model (AUC = {fusion_val_auc:.4f})', 
             linewidth=2)
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves on External Validation Set')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig('external_validation_roc.pdf')
    plt.show()

    # =============================================================================
    # 7.6 Save validation results
    # =============================================================================
    validation_results = {
        'Date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'User': 'DONGHAILU-1',
        'Clinical_AUC': clinical_val_auc,
        'Imaging_AUC': imaging_val_auc,
        'Fusion_AUC': fusion_val_auc
    }
    
    results_df = pd.DataFrame([validation_results])
    results_df.to_csv('external_validation_results.csv', index=False)
    
    with open('external_validation_report.txt', 'w') as f:
        f.write(f"External Validation Results\n")
        f.write(f"Date: {validation_results['Date']}\n")
        f.write(f"User: {validation_results['User']}\n")
        f.write("-" * 50 + "\n")
        f.write(f"Clinical Model AUC: {clinical_val_auc:.4f}\n")
        f.write(f"Imaging Model AUC: {imaging_val_auc:.4f}\n")
        f.write(f"Fusion Model AUC: {fusion_val_auc:.4f}\n")
        f.write("-" * 50 + "\n")
        f.write("\nSelected Clinical Features:\n")
        for feature in feature_names1[multistep_selected_features]:
            f.write(f"- {feature}\n")

except FileNotFoundError:
    print("Error: Could not find the external validation data files. Please make sure the file paths are correct.")
except Exception as e:
    print(f"Error during external validation: {str(e)}")

print("\nExternal validation evaluation completed.")